**Повторение**:

* зачем нужна move-семантика?
* какие есть категории выражений в С++17?
* что здесь позовётся?

```c++
class T {};

T   get_obj();
T&  get_ref();
T&& get_exp_obj();

auto x = get_obj();      // ???
auto x = get_ref();      // ???
auto x = get_exp_obj();  // ???
```

* что делает `std::move`?

* зачем здесь делать `std::move`, если у типа уже стоит `&&`?

```c++
std::string capitalize(std::string&& s)
{
    std::string rv = std::move(s);
    for (auto& c : rv)
        c = std::to_upper(c);
    return rv;
}
```

* какие бывают рекомендации по передаче параметра `s` в функцию `capitalize`?

* что здесь позовётся?

```c++
class T {};

T         get_obj();
T&        get_ref();
T&&       get_exp_obj();
const T&  get_cref();
const T&& get_exp_cobj();

auto x = std::move(get_obj());      // ???
auto x = std::move(get_ref());      // ???
auto x = std::move(get_exp_obj());  // ???
auto x = std::move(get_cref());     // ???
auto x = std::move(get_exp_cobj()); // ???
```

* что такое perfect forwaring и зачем он нужен?
* как сделать perfect forwaring для аргумента `s` в функции `capitalize`? (продиктуйте / сбросьте в чат код)

<br />

### Шаблоны. Продвинутый материал

<br />

##### Повторение основ

__Вопрос__: зачем нужны шаблоны?

__Вопрос__: что может быть параметром шаблона?

Код для повторения:

```c++
template<typename T>
void my_swap(T& x, T& y)
{
    T t = std::move(x);
    x = std::move(y);
    y = std::move(t);
}

template<>
void my_swap<string>(string& x, string& y)
{
    x.swap(y);
}
```

<br />

Шаблонный класс:
    
```c++
template<typename T>
struct Point3
{
    T x;
    T y;
    T z;
};

using Point3F = Point3<float>;
using Point3D = Point3<double>;
```

Или так:

```c++
template<typename T, int N>
struct PointN
{
    T data[N];
};

using Point3F = PointN<float, 3>;
using Point3D = PointN<double, 3>;
```

__Вопрос__: что такое частичная специализация? К чему её можно применять?

<br />

__Вопрос__: скомпилируется ли этот код?
    
```c++
template<typename T>
class Person
{
private:
    T id;
    
public:
    void set_id(T i_id) { id = std::move(i_id); }
    
    const T& get_id() const { return id; }
    
    void clear_id() { id.clear(); }
};

int main()
{
    Person<int> p;
    p.set_id(15);
    std::cout << p.get_id() << std::endl;    
}
```

<br />

__Вопрос__: как устроена компиляция и линковка шаблонов?

<br />

##### SFINAE

https://en.cppreference.com/w/cpp/language/sfinae

https://en.cppreference.com/w/cpp/language/overload_resolution

https://jguegant.github.io/blogs/tech/sfinae-introduction.html

https://www.bfilipek.com/2016/02/notes-on-c-sfinae.html

Рассмотрим пример, как через SFINAE реализовать `my_swap` так, чтобы версия с методом была вызвана для всех типов, у которых есть соответствующий метод, а не только для тех, для которых не забыли сделать специализацию:

```c++
template<typename T>
void my_swap(T& x, T& y)
{
    T t = std::move(x);
    x = std::move(y);
    y = std::move(t);
}

template<>
void my_swap<string>(string& x, string& y)
{
    x.swap(y);
}
```

SFINAE = Substitution Failure Is Not An Error

SFINAE - ovreloads resolution правило при наличии шаблонов: если не получается провести подстановку / сделать вывод шаблонных параметров, не генерировать ошибку компиляции, а продолжить искать другую подходящую перегрузку

Если речь идёт про SFINAE, то должны участвовать:
* шаблоны
* перегрузки
* поиск нужной перегрузки

Необходимые отступления:
    
* _substitution_ происходит в аргументах функции, возвращаемом типе и параметрах шаблона

    ```c++
    template<HERE>
    HERE my_function(HERE x, HERE y)
    { ...; }
    ```

* _substitution failure_ - ситуация, когда в результате подстановки в HERE получается ill-formed выражение
    * если ill-formed случился не в местах подстановки, а, например, в теле функции после подстановки, то это не случай SFINAE, а обычный hard error
    * если ill-formed случился как side-effect подстановки, то это тоже hard error (что считать side-effect-ом, что точно является sfinae error - подробнее читайте документацию): 
    
    ```c++
    // ex. 1
    template <class T, class = typename T::type>      // SFINAE failure if T has no member type
    void foo (int);

    // ex. 2
    template <typename T>
    struct B { using type = typename T::type; };

    template <class T, class = typename B<T>::type>   // hard error if T has no member type
    void foo (int);
    ```

Стандартный трюк для определения какого-нибудь свойства типа (подробно и нудно разобрать каждую строчку и почему оно работает):

```c++
// trait helper:
//     is_class<T>::value is true iff T is a class
// 
//     is_class<int>::value == false
//     is_class<string>::value == true
template<typename T>
class is_class {
    typedef char yes[1];
    typedef char no [2];
    template<typename C> static yes& test(int C::*); // selected if C is a class type
    template<typename C> static no&  test(...);      // selected otherwise
  public:
    static bool const value = sizeof(test<T>(0)) == sizeof(yes);
};
```

Демонстрация sfinae с cppreference (разобрать подробно):
    
```c++
#include <iostream>
 
// this overload is always in the set of overloads
// ellipsis parameter has the lowest ranking for overload resolution
void test(...)
{
    std::cout << "test(...)\n";
}
 
// this overload is added to the set of overloads if
// C is a reference-to-class type and F is a pointer to member function of C
template <class C, class F>
auto test(C c, F f) -> decltype((void)(c.*f)(), void())
{
    std::cout << "test(object)\n";
}
 
// this overload is added to the set of overloads if
// C is a pointer-to-class type and F is a pointer to member function of C
template <class C, class F>
auto test(C c, F f) -> decltype((void)((c->*f)()), void())
{
    std::cout << "test(pointer)\n";
}
 
struct X { void f() {} };
 
int main(){
  X x;
  test( x, &X::f);
  test(&x, &X::f);
  test(42, 1337);
}
```

<br />

Теперь мы можем написать идеальный `swap`.

```c++
template<typename T>
class has_swap_method
{
    ...
};

...
```

__Шаг 1__: напишем `class has_swap_method` по аналогии с `class is_class`, который будет отвечать на вопрос, есть ли у шаблонного параметра метод `swap` (подробно объяснить)

```c++
template<typename T>
class has_swap_method
{
    typedef char yes[1];
    typedef char no [2];
    template<typename U, void (U::*)(U&)> class S {};
    template<typename U> static yes& test(S<U, &U::swap>*);
    template<typename U> static no&  test(...);

public:
    static constexpr bool value = sizeof(test<T>(nullptr)) == sizeof(yes);
};
```

Протестируем:

```c++
#include "has_swap_method_trait.h"

#include <iostream>
#include <string>
#include <vector>

template<typename T>
void test_has_swap(const char* descr)
{
    std::cout << descr
              << " has swap method: "
              << has_swap_method<T>::value
              << std::endl;
}

struct Point
{
    float x;
    float y;
};

int main()
{
    test_has_swap<int>             ("int             ");
    test_has_swap<float>           ("float           ");
    test_has_swap<Point>           ("Point           ");
    test_has_swap<std::vector<int>>("std::vector<int>");
    test_has_swap<std::string>     ("std::string     ");

    return 0;
}
```

Вывод:

```sh
int              has swap method: 0
float            has swap method: 0
Point            has swap method: 0
std::vector<int> has swap method: 1
std::string      has swap method: 1
```

__Шаг 2__: реализация `my_swap` (почти идеально; с использованием простенького ):

```c++
template<typename T>
void my_swap(T& x, T& y, std::false_type)
{
    std::cout << "default swap\n";

    T t = std::move(x);
    x = std::move(y);
    y = std::move(t);
}

template<typename T>
void my_swap(T& x, T& y, std::true_type)
{
    std::cout << "optimized swap\n";

    x.swap(y);
}

template<typename T>
void my_swap(T& x, T& y)
{
    my_swap(x, y,
            std::integral_constant<bool, has_swap_method<T>::value>());
}
```

Протестируем:
    
```c++
int main()
{
    std::cout << "int:    ";
    int i1 = 1, i2 = 2;
    my_swap(i1, i2);

    std::cout << "string: ";
    std::string s1 = "abc", s2 = "def";
    my_swap(s1, s2);

    return 0;
}
```

Вывод:
    
```sh
int:    default swap
string: optimized swap
```

<br />

__Шаг 2__ через `std::enable_if` - лучше (объяснить):

```c++
template<typename T>
typename std::enable_if<has_swap_method<T>::value, void>::type
my_swap(T& x, T& y)
{
    std::cout << "optimized swap\n";
    x.swap(y);
}

template<typename T>
typename std::enable_if<!has_swap_method<T>::value, void>::type
my_swap(T& x, T& y)
{
    std::cout << "default swap\n";
    T t = std::move(x);
    x = std::move(y);
    y = std::move(t);
}
```

Протестируем:
    
```c++
int main()
{

    std::cout << "int:    ";
    int i1 = 1, i2 = 2;
    my_swap(i1, i2);

    std::cout << "string: ";
    std::string s1 = "abc", s2 = "def";
    my_swap(s1, s2);

    return 0;
}
```

Вывод:
    
```sh
int:    default swap
string: optimized swap
```

<br />

__Вопрос на понимание:__ почему такой вариант не скомпилируется?

```c++
template<typename T,
         void (T::*)(T&) = &T::swap>
void my_swap(T& x, T& y)
{
    std::cout << "optimized swap\n";
    x.swap(y);
}

template<typename T>
void my_swap(T& x, T& y)
{
    std::cout << "default swap\n";
    T t = std::move(x);
    x = std::move(y);
    y = std::move(t);
}

int main()
{

    std::cout << "int:    ";
    int i1 = 1, i2 = 2;
    my_swap(i1, i2);

    std::cout << "string: ";
    std::string s1 = "abc", s2 = "def";
    my_swap(s1, s2);

    return 0;
}
```

<details>
<summary>ответ</summary>
<p>
    
для `std::string` оба варианта перегрузки подходят и имеют одинаковый приоритет в механизме overloading resolution, поэтому компилятор не может сделать выбор и генерирует ошибку.

</p>
</details>

<br />

Альтернативный способ реализации `has_method_swap` с использованием `decltype`:

```c++
template<typename T>
class has_swap_method
{
    template<typename U, void (U::*)(U&)> class S {};

    template <typename C>
    static constexpr decltype(S<C, &C::swap>(), bool()) test(int /* unused */)
    {
        return true;
    }

    template <typename C>
    static constexpr bool test(...)
    {
        return false;
    }

public:
    static constexpr bool value = test<T>(int());
};
```

<br />

##### variadic templates (C++11)

https://en.cppreference.com/w/cpp/language/parameter_pack

*variadic templates* - фича стандарта С++11, позволяющая писать шаблоны с переменным числом аргументов.

Начнём сразу с демонстрационного примера: шаблонная функция записывает строку в csv-файл

```c++
template<typename T>
void printCSVLine(std::ostream& os, const T& v)
{
    os << v << std::endl;
}

template<typename T1, typename T2, typename ...Args>
void printCSVLine(std::ostream& os, const T1& v1, const T2& v2, Args&&... args)
{
    os << v1 << ',';
    printCSVLine(os, v2, args...);
}
```

Использование:

```c++
printCSVLine(std::cout, "name",     "surname",   "age", "gender");
printCSVLine(std::cout, "Добрыня",  "Никитич",   42,    'm');
printCSVLine(std::cout, "Илья"s,    "Муромец"sv, 33,    'm');
printCSVLine(std::cout, "Василиса", "Премудрая", 35,    'f');
```

<br />

Пример форматированного вывода с cppreference:
    
```c++
#include <iostream>
 
void tprintf(const char* format) // base function
{
    std::cout << format;
}
 
template<typename T, typename... Targs>
void tprintf(const char* format, const T& value, const Targs&... Fargs) // recursive variadic function
{
    for ( ; *format != '\0'; format++ ) {
        if ( *format == '%' ) {
           std::cout << value;
           tprintf(format + 1, Fargs...); // recursive call
           return;
        }
        std::cout << *format;
    }
}
 
int main()
{
    tprintf("% world% %\n","Hello",'!',123);  // Hello world! 123
}
```

<br />

Как работает `...`?

Простое объяснение: выражение слева от `...` раскрывается для каждого аргумента. В зависимости от контекста между аргументами может быть автоматически поставлена запятая (надо смотреть правила).

```c++
f(&args...);             // f(&E1, &E2, &E3)
f(n, ++args...);         // f(n, ++E1, ++E2, ++E3);
f(++args..., n);         // f(++E1, ++E2, ++E3, n);
f(h(args...) + args...); // f(h(E1,E2,E3) + E1, h(E1,E2,E3) + E2, h(E1,E2,E3) + E3)
```

<br />

##### fold expression (C++17)

https://en.cppreference.com/w/cpp/language/fold

*fold expressions* - фича стандарта С++17 - добавляет различные способы раскрытия `...` у variadic template.

Общий вид добавленных правил:
    
```c++
( pack op ... )
( ... op pack )
( pack op ... op init )
( init op ... op pack )
```

где op - бинарная/унарная правая/левая операция.

Соответственно, 4 правила развёртки:

* Unary right fold $(E op ...) ->  (E_1 op (... op (E_{N-1} op E_N)))$
* Unary left fold $(... op E) -> (((E_1 op E_2) op ...) op E_N)$
* Binary right fold $(E op ... op I) -> (E_1 op (... op (E_{N−1} op (E_N op I))))$
* Binary left fold $(I op ... op E) -> ((((I op E_1) op E_2) op ...) op E_N)$

Примеры их применения:

```c++
template<typename T, typename ...Args>
auto sum(const T& v, const Args&... args) {
    return v + ... + args;
}

sum(1, 2, 3, 4, 5, 6, 7);  // 28
sum("abc"s, "def", "ghi");  // ?
```

```c++
template<typename ...Args>
void printer(Args&&... args) {
    (std::cout << ... << args) << '\n';
}

printer("hello", "world");  // helloworld
```

<br />

##### sizeof...(args) (C++11)

Есть особый оператор `sizeof...(args)`, который возвращает число аргументов.

Рассмотрим его на примере добавления нескольких элементов в `std::vector`:

```c++
template<typename T, typename... Args>
void add_to_vector(std::vector<T>& v, Args&&... args)
{
    v.reserve(v.size() + sizeof...(args));
    (v.push_back(std::forward<Args>(args)), ...);
}

// usage:
std::vector<std::string> v;
add_to_vector(v, "Добрыня", "Илюша"s, "Алёша"sv);
add_to_vector(v, "Василиса", "Настасья"s);
```

<br />

**Замечание**: почти все примеры на variadic templates были ученическими, но они являют собой достаточно мощный механизм.

Классический пример "боевого" применения - форматирование строк `std::format`, варианты inplace конструирования `make_unique`/`make_shared`/`vector::emplace`/`optional::emplace` и т.д:

```c++
auto x = std::make_unique<std::vector<std::string>>("abc", 10); 
```

> template< class T, class... Args >
  unique_ptr<T> make_unique( Args&&... args );

```c++
std::vector<std::string> v;
v.emplace_back("a", 10);
```

> template< class... Args >
  reference emplace_back( Args&&... args );

**Вопрос**: что такое и почему && ?

<br />

##### tag dispatching && type traits

*tag dispatching* - техника выбора поведения/реализации, основыванная на типе одного из параметров (тэге) и механизме перегрузки. Обычно, этот параметр - пустая структурка.

С примером простенького tag dispatching мы уже познакомились на примере `my_swap`:

```c++
template<typename T>
void my_swap_dispatched(T& x, T& y, std::false_type)
{
    T t = std::move(x);
    x = std::move(y);
    y = std::move(t);
}

template<typename T>
void my_swap_dispatched(T& x, T& y, std::true_type)
{
    x.swap(y);
}

template<typename T>
void my_swap(T& x, T& y)
{
    my_swap_dispatched(x, y, std::integral_constant<bool, has_swap_method<T>::value>());
}
```

<br />

Традииционный пример использования tag dispatching вместе с type triants: `std::advance`

Что такое `std::advance`:

```c++
// интерфейс
template< class InputIt, class Distance >
void advance( InputIt& it, Distance n );

// использование:
std::vector<int> v = { ... };
auto vit = v.begin();
std::advance(vit, 5);  // один "прыжок"

std::list<int> l = { ... };
auto lit = l.end();
std::advance(lit, -5);  // 5 "прыжков" назад

std::forward_list<int> f = { ... };
auto fit = f.begin();
std::advance(fit, 5);  // 5 "прыжков" вперёд
```

Как его можно реализовать:

```c++
namespace std {
  namespace detail {
    template <class It, class D>
    void advance_dispatch(It& i, D n, input_iterator_tag) {
      while (n--) ++i;
    }

    template <class It, class D>
    void advance_dispatch(It& i, D n, bidirectional_iterator_tag) {
      if (n >= 0)
        while (n--) ++i;
      else
        while (n++) --i;
    }

    template <class It, class D>
    void advance_dispatch(It& i, D n, random_access_iterator_tag) {
      i += n;
    }
  }

  template <class It, class Distance>
  void advance(It& i, Distance n) {
    typename iterator_traits<It>::iterator_category category;
    detail::advance_dispatch(i, n, category);
  }
}
```

Нюанс в определении типа `iterator_traits<It>::iterator_category`

Делается это примерно так:
    
```c++
struct input_iterator_tag { };
struct bidirectional_iterator_tag { };
struct random_access_iterator_tag { };

template<typename It>
struct iterator_traits
{
    using iterator_category = input_iterator_tag;
    ...
};

template<typename T>
struct iterator_traits<std::vector<T>::iterator>
{
    using iterator_category = random_access_iterator_tag;
    ...    
};
```

После того как определён type trait для итераторов, любой алгоритм, желающий использовать преимущества random_access_iterator, может сделать это через tag dispatching.

Минус подхода в том, что для такой реализации нужно для каждого итераторане забыть прописать iterator_traits (или он свалится в самый слабый вариант traits)

<br />

##### стандартные type traits

https://en.cppreference.com/w/cpp/types

В стандартную библиотеку добавлено большое кол-во trait-ов, чтобы программисты не мучились и не писали свои велосипеды. Полный список смотрите по ссылке, а вот некоторые из них:

```c++
std::is_integral<T>
std::is_floating_point<T>
std::is_array<T>
std::is_trivial<T>
std::is_same<T, U>
```

<br />

##### deduction guides (C++17)

https://en.cppreference.com/w/cpp/language/class_template_argument_deduction

*class template argument deduction* - нововведение С++17: при использовании шаблона класса программист не обязан указывать параметры шаблона, если компилятор может вывести их из контекста (и программиста устроит то, что вывел компилятор).

```c++
std::vector v = {1, 2, 3, 4, 5};    // std::vector<int> автоматически
std::pair p = {42, 3.14};           // std::pair<int, double> автоматически
std::list l = {"hello", "world" };  // ?
```





Именно поэтому работают примеры из лекции по многопотоности:
    
```c++
std::mutex mtx;
std::lock_guard guard(mtx);  // std::lock_guard<std::mutex> guard(mtx);
```

<br />

Рассмотрим снова пример

```c++
std::vector v = {1, 2, 3, 4, 5};
```

Правило, по которому компилятор понимает, что при создании `std::vector<T>` из `std::initializer_list<int>` нужно взять `T = int`, называется *deduction guides*.

Какие-то правила прописаны по умолчанию и неявно применяются для всех классов (implicitly-generated deduction guides), какие они - читайте стандарт по ссылке.

Иногда неявных правил не хватает, и программисту хочется добавить свои. Такие правила называются пользовательскими (user defined deduction guides)

Для имеющихся контейнеров уже всё готово, поэтому предположим, у нас есть свой личный контейнер:
    
```c++
template<typename T>
class flat_set
{
public:
    flat_set(std::initializer_list<T> items);
    
    template<typename It>
    flat_set(It begin, It end);
};
```

Напишем deduction guides для нашего типа:
    
```c++
template<class It>
flat_set(It begin, It end) -> flat_set<typename std::iterator_traits<It>::value_type>;
```

Использование:

```c++
flat_set x = {1, 2, 3, 4, 5}; // flat_set<int> по неявным правилам вывода

std::vector v = {1, 2, 3, 4, 5};
flat_set x{v.begin(), v.end()};  // flat_set<int> по нашему правилу
```

<br />

**Резюме**:

* `SFINAE` - техника выбора более оптимизированной реализации на этапе компиляции по типу, пример с `my_swap` (шаблоны + перегрузки)
* variadic templates && fold expressions - шаблоны переменного числа аргументов, необходимый механизм для функций `make_unique`, `make_shared`, `emplace_back`.
* deduction guides - способ автоматического вывода типов у шаблонов классов через выражение справа.

<br />